<a href="https://colab.research.google.com/github/eshaandinesh/Colab/blob/main/Assignment1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import files


uploaded = files.upload()

Saving CarPrice_Assignment.csv to CarPrice_Assignment.csv


In [ ]:
!pip install -q findspark
!pip install pyspark
import os
import findspark
import sys



     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.9/316.9 MB 4.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.0-py2.py3-none-any.whl size=317425344 sha256=b5d4d4d7f6e83c4ac0ebbeb53c18453a861db8253b435ca9891d7a62a3ad79c2
  Stored in directory: /root/.cache/pip/wheels/41/4e/10/c2cf2467f71c678cfc8a6b9ac9241e5e44a01940da8fbb17fc
Successfully built pyspark


In [ ]:
from pyspark.sql import SparkSession
from pyspark.ml.feature import StringIndexer, VectorAssembler
from pyspark.ml.regression import LinearRegression
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml import Pipeline

from collections import OrderedDict
import numpy as np


# Create a SparkSession
spark = SparkSession.builder.appName("LinearRegression").getOrCreate()

# Load the data
df = spark.read.format("csv").option("header", "true").option("inferSchema", "true").load("CarPrice_Assignment.csv")

# Assuming that 'price' is your target variable and the rest are features
feature_columns = df.columns  # list of column names excluding target
feature_columns.remove("price")  # replace "price" with your actual target column

# Convert categorical variable into dummy/indicator variables.
indexers = [StringIndexer(inputCol=column, outputCol=column+"_index").fit(df) for column in feature_columns]
df = Pipeline(stages=indexers).fit(df).transform(df)

# Assemble the features into a single vector
assembler = VectorAssembler(inputCols=[column+"_index" for column in feature_columns], outputCol="features")
df = assembler.transform(df)

# Split the data into training and test sets
train_data, test_data = df.randomSplit([0.8, 0.2])

# Create a linear regression model
lr = LinearRegression(featuresCol='features', labelCol='price')

# Fit the model to the training data
lr_model = lr.fit(train_data)

# Make predictions on the test set
predictions = lr_model.transform(test_data)

# Evaluate the model using RMSE (Root Mean Squared Error)
evaluator = RegressionEvaluator(predictionCol="prediction", labelCol="price", metricName="rmse")
rmse = evaluator.evaluate(predictions)

print(f'Root Mean Squared Error: {rmse}')

# Print the coefficients and intercept for linear regression
print("Coefficients: " + str(lr_model.coefficients))
print("Intercept: " + str(lr_model.intercept))

# Summarize the model over the training set and print out some metrics
trainingSummary = lr_model.summary
print("R-squared: %f" % trainingSummary.r2)

# Get the feature names from the VectorAssembler
feature_names = assembler.getInputCols()

# Get the coefficients from the LinearRegressionModel
coefficients = lr_model.coefficients


d = dict()
# Print the coefficients along with the feature names
for coef, feat in zip(coefficients, feature_names):
    #print(f'Feature: {feat}, Coefficient: {coef}')
    d[feat] = coef


d_sorted = dict(sorted(d.items(), key = lambda x: x[1], reverse = True))
#print(d_sorted)

for f in d_sorted:
    print(f[:-6:1],d_sorted[f])

Root Mean Squared Error: 2638.4924229723974
Coefficients: [6.471054111520213,-141.98370610458926,-14.988257909697616,123.27751165846001,2110.3926709573,-1229.762771726163,-71.29240692720094,2005.34193124227,17975.863153630344,-43.82987404941299,86.04813114133854,154.95796455062367,-46.67145269148587,31.690678044386008,-201.17814990105012,1350.6117666341815,129.13342815622212,-1260.794151227456,107.51632866707514,12.393775275914575,159.9010373109647,-51.51798620812103,106.29554246560254,37.11538608117554,-19.00528465508206]
Intercept: 5614.5294977811345
R-squared: 0.749274
enginelocation 17975.863153630344
aspiration 2110.3926709573
drivewheel 2005.34193124227
cylindernumber 1350.6117666341815
compressionratio 159.9010373109647
carwidth 154.95796455062367
enginesize 129.13342815622212
fueltype 123.27751165846001
boreratio 107.51632866707514
peakrpm 106.29554246560254
carlength 86.04813114133854
citympg 37.11538608117554
curbweight 31.690678044386008
stroke 12.393775275914575
car_ID 6.47